In [3]:
#Video to frames

import cv2
import numpy as np
from IPython.display import Image

def getallframes():
    frames=[]
    cap=cv2.VideoCapture('C:/Users/paliw/Desktop/Dataset/Videos/ICC WT20 - England v Sri Lanka Match Highlights.avi')
    count=0
    ret=1
    while(ret):
        ret,frame=cap.read()
        if ret==False:
            break
        cv2.imwrite("C:/Users/paliw/Desktop/img/"+str(count)+'.png',frame)
        count +=1
    cap.release()
    cv2.destroyAllWindows()
     

#from IPython.display import display, Image


getallframes()

In [2]:
#list of frames with batsman

f = open("C:/Users/paliw/Desktop/Batsman_Dataset/annotation.txt", "r")
f1 = f.readlines()
ids = []
for x in f1:
    idx = x.split(",")
    ids.append(idx[0])
#print(ids)

['98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', '285', '286', '287', '288', '289', '290', '291', '292', '293', '294', '295', '296', '297', '298', '299', '300', '301', '302', '303', '304', '305', '306', '307', '308', '309', '310', '311', '312', '313', '314', '315', '316', '317', '404', '405', '406', '407', '408', '409', '410', '411', '412', '413', '414', '415', '416', '417', '418', '419', '420', '421', '422', '423', '424', '425', '426', '427', '428', '429', '430', '431', '432', '433', '434', '435', '521', '522', '523', '524', '525', '526', 

In [15]:
#Shortlisting frames with batsman

import os 
import shutil

all_fr = sorted(os.listdir("C:/Users/paliw/Desktop/img"))
dst = "C:/Users/paliw/Desktop/Batsman_Dataset/frames"
for fr in all_fr:
    f = fr.split(".")
    if f[0] in ids:
        src = os.path.join("C:/Users/paliw/Desktop/img", fr)
        shutil.copy(src, dst)

In [21]:
#Writing custom dataset for Batsman Detection

import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image


class BatsmanDetectionDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "frames"))))
        self.bbox = os.path.join(root, "annotation.txt")

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "frames", self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        
        fr = self.imgs[idx].split(".")
        fr_id = fr[0]
        
        num_objs = 1
        boxes = []
        #bbox coordinates
        f = open(self.bbox, "r")
        f1 = f.readlines()
        for x in f1:
            crd = x.split(",")
            if fr_id == crd[0]:
                xmin = int(crd[2])
                ymin = int(crd[3])
                xmax = int(crd[4])
                ymax = int(crd[5])
                boxes.append([xmin, ymin, xmax, ymax])
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        frame_id = torch.tensor([int(fr_id)])
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["frame_id"] = frame_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [23]:
dataset = BatsmanDetectionDataset("C:/Users/paliw/Desktop/Batsman_Dataset")
dataset[100]

(<PIL.Image.Image image mode=RGB size=640x360 at 0x24D07B12748>,
 {'boxes': tensor([[298.,  46., 372., 169.]]),
  'labels': tensor([1]),
  'image_id': tensor([100]),
  'frame_id': tensor([1289]),
  'area': tensor([9102.]),
  'iscrowd': tensor([0])})